### Import all of the needed libraries

In [ ]:
# YOU ARE GOING TO HAVE TO UPLOAD THE REQUIEMENTS.TXT HERE
#apparently, the UI is broken in colab :/
from google.colab import files
import pkg_resources

required_pkgs=["gensim","vaderSentiment"]
installed_packages = pkg_resources.working_set
installed_package_names = list(map(lambda item : item.project_name, installed_packages))

if any(pkg in required_pkgs for pkg in installed_package_names) :
  print("""
  -------------------------------
  All packages installed!
  -------------------------------
  """)
else :
  print("""
  /////////////////////////////////////////////
  Please upload supplied requirements.txt file.
  /////////////////////////////////////////////
  """)
  files.upload()
  print("""
  ==================================================================================================
  NOTE: You will need to restart the session anc click "Run All" again after all packeges download.
  ==================================================================================================
  """)

/tmp/ipython-input-424614548.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources



  -------------------------------
  All packages installed!
  -------------------------------
  


In [ ]:
!pip install -r requirements.txt

In [ ]:
from tensorflow.keras import layers, Sequential
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from transformers import pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from tensorflow import keras
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import F1Score
from zipfile import ZipFile
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, AdamW
from tensorflow.keras.utils import Sequence
from google.colab import drive
from sklearn.utils import resample
import os
import gdown
import cv2
import random
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import re
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

#Check if the drive is present just in case
drive_path = '/content/drive/MyDrive/'
if os.path.exists(drive_path):
    drive.flush_and_unmount()
else:
    print("Drive not found.")

#Next, extract the csv file from googe drive
if os.path.exists("banking_complaints.csv") == False :
  url = "https://drive.google.com/uc?id=1dHYxuc9LdWv44bDzycdIpK1AtjVVe2UX"
  output = "banking_complaints.csv"
  gdown.download(url, output)

complaints_df = pd.read_csv("banking_complaints.csv")
complaints_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Drive not found.


Downloading...
From: https://drive.google.com/uc?id=1dHYxuc9LdWv44bDzycdIpK1AtjVVe2UX
To: /content/banking_complaints.csv
100%|██████████| 9.01M/9.01M [00:00<00:00, 49.3MB/s]


,Complaint ID,Date Received,Banking Product,Issue ID,Complaint Description,State,ZIP,Bank Response
0,CID76118977,1/1/2023,Checking or savings account,I_3510635,on XX/XX/XX22 I opened a safe balance account ...,California,92311,Closed with monetary relief
1,CID98703933,1/1/2023,"Credit reporting, credit repair services, or o...",I_3798538,There is an item from Bank of ABC on my credit...,California,91344,Closed with explanation
2,CID52036665,1/1/2023,Checking or savings account,I_3648593,On XX/XX/XX22 I found out that my account was ...,New York,10466,Closed with monetary relief
3,CID62581335,1/1/2023,Credit card or prepaid card,I_6999080,I've had a credit card for years with Bank of ...,California,92127,Closed with monetary relief
4,CID65731164,1/1/2023,Checking or savings account,I_3648593,This issue has to do with the way that Bank of...,New Jersey,7946,Closed with explanation
...,...,...,...,...,...,...,...,...
7006,CID63720677,7/6/2023,Mortgage,I_2806126,I spoke with XXXX XXXX of Capital One in XX/XX...,New York,114XX,Closed with explanation
7007,CID93313606,7/6/2023,Mortgage,I_4197360,1. PNC customer service & the letter I receive...,Ohio,430XX,Closed with explanation
7008,CID13952171,7/6/2023,Mortgage,I_4197360,We went to refinance with another company due ...,Maryland,212XX,Closed with explanation
7009,CID32570168,7/6/2023,Credit reporting,I_9082488,After XXXX letters mailed to Transunion via ce...,Idaho,836XX,Closed with explanation


### EDA

In [ ]:
complaints_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7011 entries, 0 to 7010
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Complaint ID           7011 non-null   object
 1   Date Received          7011 non-null   object
 2   Banking Product        7011 non-null   object
 3   Issue ID               7011 non-null   object
 4   Complaint Description  7011 non-null   object
 5   State                  6984 non-null   object
 6   ZIP                    6981 non-null   object
 7   Bank Response          7011 non-null   object
dtypes: object(8)
memory usage: 438.3+ KB


#### Get the total amount of values that are null and see which ones are null.

In [ ]:
complaints_df.isna().sum()

,0
Complaint ID,0
Date Received,0
Banking Product,0
Issue ID,0
Complaint Description,0
State,27
ZIP,30
Bank Response,0


### View the rows with null values

In [ ]:
complaints_df.loc[(complaints_df["ZIP"].isna())|(complaints_df["State"].isna())]

,Complaint ID,Date Received,Banking Product,Issue ID,Complaint Description,State,ZIP,Bank Response
122,CID70242892,1/10/2023,Checking or savings account,I_5781460,Bank of ABC kept deducting {$5.00} from my sav...,NaN,NaN,Closed with monetary relief
513,CID70113079,2/8/2023,Checking or savings account,I_5781460,Complaint number : XXXX. \nI am reopening my l...,NaN,NaN,Closed with explanation
810,CID66548469,3/1/2023,"Money transfer, virtual currency, or money ser...",I_6729142,XX/XX/XXXX Dear Consumer Financial Protection ...,Florida,NaN,Closed with explanation
847,CID58104772,3/4/2023,"Money transfer, virtual currency, or money ser...",I_1514870,First name : XXXX XXXX Last name : XXXX Addres...,New York,NaN,Closed with explanation
944,CID76413078,3/10/2023,"Money transfer, virtual currency, or money ser...",I_5763596,Im trying to exchange damaged bills in differe...,NaN,NaN,Closed with explanation
1091,CID62753476,3/22/2023,Mortgage,I_9057781,"Bank of ABC, loan officer, XXXX XXXX XXXX, sti...",NaN,19141,Closed with explanation
1113,CID53371787,3/23/2023,Credit card or prepaid card,I_6355741,I WAS PROVIDED WITH A XXXX GIFT CARD FOR {$10...,NaN,NaN,Closed with explanation
1206,CID85269461,3/29/2023,Checking or savings account,I_3648593,"Hi, I am XXXX and currently in XXXX XXXX for l...",NaN,NaN,Closed with explanation
1359,CID94119891,4/7/2023,Checking or savings account,I_3510635,I have informed Bank of ABC on XX/XX/XXXX that...,NaN,NaN,Closed with monetary relief
1456,CID59980598,4/14/2023,Credit card or prepaid card,I_7000941,So I have a BofA Cash reward card and a Travel...,NaN,NaN,Closed with explanation


#### After looking at the rows that have null values, I think it is safe to go ahead and drop them since their number is pretty small.

In [ ]:
complaints_df = complaints_df.dropna()
complaints_df

,Complaint ID,Date Received,Banking Product,Issue ID,Complaint Description,State,ZIP,Bank Response
0,CID76118977,1/1/2023,Checking or savings account,I_3510635,on XX/XX/XX22 I opened a safe balance account ...,California,92311,Closed with monetary relief
1,CID98703933,1/1/2023,"Credit reporting, credit repair services, or o...",I_3798538,There is an item from Bank of ABC on my credit...,California,91344,Closed with explanation
2,CID52036665,1/1/2023,Checking or savings account,I_3648593,On XX/XX/XX22 I found out that my account was ...,New York,10466,Closed with monetary relief
3,CID62581335,1/1/2023,Credit card or prepaid card,I_6999080,I've had a credit card for years with Bank of ...,California,92127,Closed with monetary relief
4,CID65731164,1/1/2023,Checking or savings account,I_3648593,This issue has to do with the way that Bank of...,New Jersey,7946,Closed with explanation
...,...,...,...,...,...,...,...,...
7006,CID63720677,7/6/2023,Mortgage,I_2806126,I spoke with XXXX XXXX of Capital One in XX/XX...,New York,114XX,Closed with explanation
7007,CID93313606,7/6/2023,Mortgage,I_4197360,1. PNC customer service & the letter I receive...,Ohio,430XX,Closed with explanation
7008,CID13952171,7/6/2023,Mortgage,I_4197360,We went to refinance with another company due ...,Maryland,212XX,Closed with explanation
7009,CID32570168,7/6/2023,Credit reporting,I_9082488,After XXXX letters mailed to Transunion via ce...,Idaho,836XX,Closed with explanation


### Test for duplicates

In [ ]:
print("TOTAL DUPLICATE REVIEWS => " + str(complaints_df.duplicated().sum()))
complaints_df[complaints_df.duplicated()]

TOTAL DUPLICATE REVIEWS => 0


,Complaint ID,Date Received,Banking Product,Issue ID,Complaint Description,State,ZIP,Bank Response


#### There are no duplicates which is good.

#### Check if the variables have correct datatypes. Make changes wherever necessary.

In [ ]:
complaints_df[complaints_df.columns].dtypes

,0
Complaint ID,object
Date Received,object
Banking Product,object
Issue ID,object
Complaint Description,object
State,object
ZIP,object
Bank Response,object


#### We now need to convert the dates into datetimes

In [ ]:
complaints_df["Date Received"] = pd.to_datetime(complaints_df["Date Received"])

/tmp/ipython-input-3960315267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complaints_df["Date Received"] = pd.to_datetime(complaints_df["Date Received"])


### Get the date range

In [ ]:
date_range = pd.date_range(start=complaints_df["Date Received"].min(), end=complaints_df["Date Received"].max(), freq='D')
date_range

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10',
               ...
               '2023-10-12', '2023-10-13', '2023-10-14', '2023-10-15',
               '2023-10-16', '2023-10-17', '2023-10-18', '2023-10-19',
               '2023-10-20', '2023-10-21'],
              dtype='datetime64[ns]', length=294, freq='D')

### Define a function named preprocessing that executes the following series of pre-processing steps in order:
- Convert text to lowercase
- Remove numbers
- Remove stopwords
- Remove punctuation
- Apply lemmatization

In [ ]:
def preprocessing(text_list):
  reviews = list(text_list.values.astype(str))

  #make words lower case
  lower_case_reviews = []
  for tok_review in text_list  :
    tokenized_review = tok_review.lower()
    lower_case_reviews.append(tokenized_review)

  #Remove numbers
  removed_digits = []
  for tok_review in lower_case_reviews:
    new_review = re.sub(r"[0-9].*?"," ", tok_review)
    removed_digits.append(new_review)

  #remove stop words
  no_stop_words_reviews = []
  for tok_review in removed_digits:
    stop_words = nltk.corpus.stopwords.words('english')
    stop_word_space = ["\s" + word + "\s" for word in stop_words]
    stop_word_regex = "|".join(stop_word_space)
    new_review = re.sub(r""+stop_word_regex+""," ", tok_review)
    no_stop_words_reviews.append(new_review)

  #remove punctuation
  no_punctuation_reviews = []
  for tok_review in no_stop_words_reviews:
    punctuation = list(string.punctuation)
    punctuation_regex = "|\\".join(punctuation)
    new_review = re.sub(r""+punctuation_regex+""," ", tok_review)
    no_punctuation_reviews.append(new_review)

  #tokenize words
  tokenized_reviews = []
  for review in no_punctuation_reviews:
    new_review = nltk.word_tokenize(review)
    tokenized_reviews.append(new_review)

  #lemmatize words
  lemmatized_words = []
  lemmatizer = nltk.stem.WordNetLemmatizer()
  for tok_review in tokenized_reviews:
    new_lematized_array = []
    for word in tok_review:
      lemmatized_word = lemmatizer.lemmatize(word)
      new_lematized_array.append(lemmatized_word)
    lemmatized_words.append(new_lematized_array)

  return tokenized_reviews


### Clean the text under ‘Complaint Description’ using the above function

In [ ]:
final_text_list = preprocessing(complaints_df["Complaint Description"])

### Put the cleaned text into a new dataframe

In [ ]:
def combine_tokens_to_string(tokens):
  combined_tokens = []
  for token in tokens :
    combined_tokens.append(" ".join(token))
  return combined_tokens

#Complaint ID,	Date Received,	Banking Product,	Issue ID,	Complaint Description,	State,	ZIP,	Bank Response

complaints_model_df = pd.DataFrame({
    "Complaint Description":combine_tokens_to_string(final_text_list),
    "Banking Product":complaints_df["Banking Product"]
})

complaints_model_df

,Complaint Description,Banking Product
0,on xx xx xx opened safe balance account online...,Checking or savings account
1,there an item bank abc my credit report does b...,"Credit reporting, credit repair services, or o..."
2,on xx xx xx found that account frozen no appar...,Checking or savings account
3,i ve a credit card years bank abc xx xx xxxx p...,Credit card or prepaid card
4,this issue to with way bank abc their account ...,Checking or savings account
...,...,...
7006,i spoke xxxx xxxx capital one xx xx xxxx expla...,Mortgage
7007,pnc customer service letter receive contradict...,Mortgage
7008,we went refinance another company due the xxxx...,Mortgage
7009,after xxxx letters mailed transunion via certi...,Credit reporting


#### Build department names based off of information provided by Banking Product column via feature engineering

In [ ]:
classes = set(complaints_df["Banking Product"].unique())
classes

#Theres got to be a better way to do this.
department = {
    "Bank account or service" : "Account Management",
    "Checking or savings account" : "Account Management",
    "Consumer Loan" : "Loans Department",
    "Credit card" : "Account Management",
    "Credit card or prepaid card" : "Account Management",
    "Credit reporting" : "Credit Administration Department",
    "Credit reporting, credit repair services, or other personal consumer reports" : "Credit Administration Department",
    "Debt collection": "Collections Department",
    "Money transfer, virtual currency, or money service":"Compliance department",
    "Money transfers":"Operations Department",
    "Mortgage":"Loans",
    "Other financial service" : "Account Management",
    "Payday loan" : "Loans Department",
    "Payday loan, title loan, or personal loan" : "Loans Department",
    "Prepaid card" : "Account Management",
    "Student loan" : "Loans Department",
    "Vehicle loan or lease" : "Vehicle loan or lease"
}

complaints_model_df["Department"] = complaints_model_df["Banking Product"].map(department)
complaints_model_df

,Complaint Description,Banking Product,Department
0,on xx xx xx opened safe balance account online...,Checking or savings account,Account Management
1,there an item bank abc my credit report does b...,"Credit reporting, credit repair services, or o...",Credit Administration Department
2,on xx xx xx found that account frozen no appar...,Checking or savings account,Account Management
3,i ve a credit card years bank abc xx xx xxxx p...,Credit card or prepaid card,Account Management
4,this issue to with way bank abc their account ...,Checking or savings account,Account Management
...,...,...,...
7006,i spoke xxxx xxxx capital one xx xx xxxx expla...,Mortgage,Loans
7007,pnc customer service letter receive contradict...,Mortgage,Loans
7008,we went refinance another company due the xxxx...,Mortgage,Loans
7009,after xxxx letters mailed transunion via certi...,Credit reporting,Credit Administration Department


### Convert the pre-processed text into a matrix of TF-IDF features for downstream modelling after splitting up the data for training.

In [ ]:
from scipy.sparse import hstack

X = complaints_model_df[["Banking Product", "Complaint Description"]]
y = complaints_model_df["Department"]

cat_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_cat = cat_encoder.fit_transform(complaints_model_df["Banking Product"].values.reshape(-1, 1))

text_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))
X_text = text_vectorizer.fit_transform(complaints_model_df["Complaint Description"])

X = hstack([X_text, X_cat])
y = complaints_model_df["Department"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#### In order to effectively manage the process, it is critically important to categorise the complaint and pass on to the concerned product department. Consider the department as a target variable and build a classification model.

### Build a balanced random forest model to identify the correct department for a complaint (NOTE: this will take a bit to train, but it gets the best results due to built-in regularization.)

In [ ]:
nb_model = BalancedRandomForestClassifier(
    n_estimators=290,
    random_state=42,
    n_jobs=-1,
  )
grid2 = GridSearchCV(
    estimator=nb_model,
    param_grid={},
    cv=5,
)

grid2.fit(X_train, y_train)
best_model2 = grid2.best_estimator_
nb_y_pred = best_model2.predict(X_test)
report = classification_report(y_test, nb_y_pred)
print(report)
cm = confusion_matrix(y_test, nb_y_pred)
print("CONFUSION MATRIX : \n", cm)

                                  precision    recall  f1-score   support

              Account Management       0.99      0.83      0.90       701
          Collections Department       0.84      0.99      0.91       150
           Compliance department       0.71      0.99      0.83        80
Credit Administration Department       0.88      0.95      0.91       222
                           Loans       0.97      1.00      0.98       169
                Loans Department       0.81      0.92      0.86        65
           Operations Department       0.12      1.00      0.21         2
           Vehicle loan or lease       0.70      1.00      0.82         7

                        accuracy                           0.90      1396
                       macro avg       0.75      0.96      0.80      1396
                    weighted avg       0.92      0.90      0.91      1396

CONFUSION MATRIX : 
 [[582  26  31  28   5  14  15   0]
 [  1 148   0   0   1   0   0   0]
 [  0   0  79   1 

### Use SentimentIntensityAnalyzer  to predict sentiments from the complaints. The SentimentIntensityAnalyzer is a class from the vaderSentiment library designed for sentiment analysis. It evaluates text to determine the sentiment scores across four categories: positive, negative, neutral, and compound. The compound score is a normalized value between -1 (most extreme negative) and +1 (most extreme positive), providing an overall sentiment rating. This analyzer is particularly effective for social media and other informal texts, as it can interpret emoticons, acronyms, and slang. It is widely used for tasks like sentiment classification, opinion mining, and customer feedback analysis. Its ease of use and accuracy make it a valuable tool in NLP.

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(val):
  vs_val = analyzer.polarity_scores(val)['compound']
  if -0.3 < vs_val <= 0 :
    return "Low"
  elif -0.5 < vs_val <= -0.3:
    return "Medium"
  elif -0.7 < vs_val <= -0.5:
    return "High"
  else:
    return "Urgent"

complaints_df["Department"] = complaints_model_df["Department"]
complaints_df["Complaint Severity"] = complaints_model_df["Complaint Description"].apply(get_sentiment)
complaints_df.to_csv('complaints_final.csv', index=False)
complaints_df

/tmp/ipython-input-3124539561.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complaints_df["Department"] = complaints_model_df["Department"]
/tmp/ipython-input-3124539561.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complaints_df["Complaint Severity"] = complaints_model_df["Complaint Description"].apply(get_sentiment)


,Complaint ID,Date Received,Banking Product,Issue ID,Complaint Description,State,ZIP,Bank Response,Department,Complaint Severity
0,CID76118977,2023-01-01,Checking or savings account,I_3510635,on XX/XX/XX22 I opened a safe balance account ...,California,92311,Closed with monetary relief,Account Management,Urgent
1,CID98703933,2023-01-01,"Credit reporting, credit repair services, or o...",I_3798538,There is an item from Bank of ABC on my credit...,California,91344,Closed with explanation,Credit Administration Department,Low
2,CID52036665,2023-01-01,Checking or savings account,I_3648593,On XX/XX/XX22 I found out that my account was ...,New York,10466,Closed with monetary relief,Account Management,High
3,CID62581335,2023-01-01,Credit card or prepaid card,I_6999080,I've had a credit card for years with Bank of ...,California,92127,Closed with monetary relief,Account Management,Urgent
4,CID65731164,2023-01-01,Checking or savings account,I_3648593,This issue has to do with the way that Bank of...,New Jersey,7946,Closed with explanation,Account Management,Urgent
...,...,...,...,...,...,...,...,...,...,...
7006,CID63720677,2023-07-06,Mortgage,I_2806126,I spoke with XXXX XXXX of Capital One in XX/XX...,New York,114XX,Closed with explanation,Loans,Urgent
7007,CID93313606,2023-07-06,Mortgage,I_4197360,1. PNC customer service & the letter I receive...,Ohio,430XX,Closed with explanation,Loans,Urgent
7008,CID13952171,2023-07-06,Mortgage,I_4197360,We went to refinance with another company due ...,Maryland,212XX,Closed with explanation,Loans,Urgent
7009,CID32570168,2023-07-06,Credit reporting,I_9082488,After XXXX letters mailed to Transunion via ce...,Idaho,836XX,Closed with explanation,Credit Administration Department,Urgent


### How can the score be used by the bank? Share your insights.
---
Since the complaints are always going to be "negative" the best use for sentiment analysis would be to categorize the severity of the complaint. The higher the negativity score, the more likely the user will switch banks as demonstrated in the above dataframe.

---
Final thoughts and Report
---

The amount of urgent requests that we are getting is staggering. Action to resolve these requests need to be taken as soon as humanly possible. here are the totals for each relivant value for complaint severity.

In [ ]:
complaints_df["Complaint Severity"].value_counts()

,count
Complaint Severity,
Urgent,5160
Low,755
High,583
Medium,481


As you can see from the data above, the amount of urgent complaints drastically overruns the rest of the severity classes. Below I am going to go through which departments are responsible for most of these urgent classifications.

In [ ]:
complaints_df.groupby("Department")["Complaint Severity"].value_counts()

Department                        Complaint Severity
Account Management                Urgent                2631
                                  Low                    349
                                  High                   276
                                  Medium                 247
Collections Department            Urgent                 525
                                  Low                     83
                                  High                    81
                                  Medium                  59
Compliance department             Urgent                 294
                                  Low                     38
                                  High                    37
                                  Medium                  29
Credit Administration Department  Urgent                 825
                                  Low                    142
                                  High                    80
                                  Medium                  62
Loans                             Urgent                 613
                                  Low                     99
                                  High                    72
                                  Medium                  63
Loans Department                  Urgent                 240
                                  Low                     39
                                  High                    31
                                  Medium                  18
Operations Department             Urgent                   5
                                  Low                      4
                                  High                     1
                                  Medium                   1
Vehicle loan or lease             Urgent                  27
                                  High                     5
                                  Medium                   2
                                  Low                      1
Name: count, dtype: int64

Based on the data above. Account management seems to be the most prominent culprit, and is therefore due for a tactful departmental audit of their policies, performance, work environment, and more importantly, customer experience. Should this department undergo correction, that negates 51% of our urgent complaints for the entire bank.


In conclusion:
---

It is understandable that the account management department can be the most stressful and difficult department to work in, but nevertheless, we need to find ways to boost performance here. A meeting with the VP of the department should be schedule in order to further iron out the best course of action in order to enact correctional measuers based off of the data given.

How this can be improved later:

- For our random our balanced forest classifier, we can perfrom semantic-vased analysis with wor2vec or better yet, doc2vec in order to capture overall meaning of each document in order to increase the f1 and precision scores of the minority classes for increased score regardless of how often they appear on the dataset. All-in all though, doc2vec would be the better choice because if we used word2vec, we would have to get the average of all the words for each document in the corpus to determine semantic meaning.

- Perhaps later on down the road the balanced random forest model can also be fine-tuned for speed a bit better since it does take a bit to train by finding a sleeker hyperparameter combination, OR using a different model entirely.

- Another possible approach later on is to use some topic modeling along with vader sentiment in order to improve the accuracy of the classification of complaints, because vader is good at determining severity via emotional tone, but it needs more nuanced aid with these types of complaints.